In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
from semopy import Model, Optimizer
import matplotlib.pyplot as plt
from semopy import Model, Optimizer, semplot

In [2]:
def get_adj_close(tickets, start=None, end=None):
    return yf.download(tickets, start=start, end=end)['Adj Close']

In [3]:
tickers = ['BIMBOA.MX', 'KHC', 'NESN.SW', 'PEP', 'KOF', 'MKC', 'HERDEZ.MX', 'BN.PA', 'ULVR.L', 'HLF', 'CAG', 'MDLZ']
closes = get_adj_close(tickers, start='2018-01-01', end='2024-01-01')
rend = closes.pct_change()
rend = rend.dropna()
rend.head()

[*********************100%%**********************]  12 of 12 completed


Ticker,BIMBOA.MX,BN.PA,CAG,HERDEZ.MX,HLF,KHC,KOF,MDLZ,MKC,NESN.SW,PEP,ULVR.L
Date,,,,,,,,,,,,
2018-01-04,0.006221,0.014835,0.016341,-0.006970,0.009734,0.011686,0.003143,0.002348,0.006307,-0.000959,0.004926,0.003080
2018-01-05,-0.002977,0.018308,0.003163,0.023909,0.011079,-0.000899,0.008829,0.012887,0.009793,0.005282,0.002874,0.005281
2018-01-08,-0.000689,-0.006690,-0.014189,0.000000,-0.007400,0.008222,0.010023,-0.005089,0.001067,-0.001194,-0.005730,-0.000244
2018-01-09,0.002758,-0.001403,-0.003464,-0.009426,0.007455,0.001784,-0.009224,-0.000465,-0.000969,0.009087,0.000763,-0.001833
2018-01-10,-0.012835,-0.029928,-0.014977,-0.024222,-0.019496,-0.009285,-0.016787,-0.008374,-0.014934,-0.014218,-0.005081,-0.019221


In [4]:
model_desc = f"""
# Latent Variables
Mood =~ {tickers[0]} + {tickers[1]} + {tickers[2]} + {tickers[3]} + {tickers[4]} + {tickers[5]} + {tickers[6]} + {tickers[7]} + {tickers[8]} + {tickers[9]} + {tickers[10]} + {tickers[11]}
"""

In [5]:
mod = Model(model_desc)
res_opt = mod.fit(rend)
estimates = mod.inspect()

# Imprimir los resultados del ajuste del modelo
print(estimates)

         lval  op       rval      Estimate      Std. Err    z-value   p-value
0   BIMBOA.MX   ~       Mood  1.000000e+00             -          -         -
1         KHC   ~       Mood  1.054822e-01  33116.675655   0.000003  0.999997
2     NESN.SW   ~       Mood  8.697261e-02  78612.706711   0.000001  0.999999
3         PEP   ~       Mood  1.197560e-01  50575.167131   0.000002  0.999998
4         KOF   ~       Mood  2.654509e-01  22887.411058   0.000012  0.999991
5         MKC   ~       Mood  1.075960e-01  45732.648477   0.000002  0.999998
6   HERDEZ.MX   ~       Mood  2.006655e-01  13611.285021   0.000015  0.999988
7       BN.PA   ~       Mood  9.109254e-02    62314.1952   0.000001  0.999999
8      ULVR.L   ~       Mood  9.542177e-02  64045.607019   0.000001  0.999999
9         HLF   ~       Mood  1.310801e-01   5058.483379   0.000026  0.999979
10        CAG   ~       Mood  8.027511e-02  43488.525552   0.000002  0.999999
11       MDLZ   ~       Mood  1.410864e-01   43029.98295   0.000

In [6]:
# Paso 5: Visualizar el Modelo SEM con un Diagrama de Trayectoria
print(res_opt)

Name of objective: MLW
Optimization method: SLSQP
Optimization successful.
Optimization terminated successfully
Objective value: 92.059
Number of iterations: 27
Params: 0.105 0.087 0.120 0.265 0.108 0.201 0.091 0.095 0.131 0.080 0.141 3.144 9.170 4.459 0.311 0.001 2.562 0.000 4.322 4.917 0.000 14.612 6.012 9.686


In [8]:
mood_factors = res_opt.x[0:(len(tickers)-1)].tolist()
mood_factors.insert(0, 1)
mood_factors

[1,
 0.10548223664052456,
 0.08697260964858426,
 0.11975596722136188,
 0.26545085806012075,
 0.10759601711505727,
 0.20066548609220447,
 0.0910925402659696,
 0.09542176508944537,
 0.13108007388622034,
 0.08027511395157827,
 0.14108638644571353]

In [9]:
def calculate_latent_values(df, factors, var):
    resultado = pd.DataFrame()
    for i in range(len(factors)):
        resultado[var[i]] =  df.iloc[:, i] * factors[i]
    resultado['Latent Value'] = resultado.sum(axis=1)
    resultado = resultado['Latent Value']
    return resultado

In [10]:
mood = calculate_latent_values(rend[tickers], mood_factors, tickers)
mood

Date
2018-01-04    0.012639
2018-01-05    0.011623
2018-01-08   -0.001296
2018-01-09   -0.000286
2018-01-10   -0.036083
                ...   
2023-12-22    0.009135
2023-12-26    0.000291
2023-12-27   -0.003248
2023-12-28   -0.005333
2023-12-29   -0.013226
Name: Latent Value, Length: 1550, dtype: float64

In [11]:
g = semplot(mod, 'pd.png')

## Gráfica

<img src="pd.png">